In [54]:
import os
import tqdm
import wandb
import warnings
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

from matplotlib.axes import Axes
from wandb.apis.public import Run

from typing import Union, List, Dict
from src.visualization import set_themes

api = wandb.Api() # Initialize Weights & Biases API, used for fetching run data
set_themes() # Set custom themes for plots
pl.Config.set_tbl_rows(20) # Set Polars table display rows limit

pd.set_option('future.no_silent_downcasting', True)

failed to send, dropping 2 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 31 additional messages skipped


In [55]:
config = {
    "model": "matrix_factorization",
    "ensure_available_locally": False
}
sorting_criterion = {
    "epoch/test_hitrate@50": 0.5,
    "epoch/test_ndcg@50": 0.25,
}

def fetch_run_metadata(run: Run, considered_metrics: Union[str, Dict[str, float]] = "epoch/epoch") -> Dict:
    run_config = {}
    for key, value in run.config.items():
        # Convert lists and dicts to strings
        if isinstance(value, (list, dict)):
            run_config[key] = str(value)
        else:
            run_config[key] = value

    run_history = run.history()
    run_history = run_history.replace({"Infinity": np.inf, "NaN": np.nan})

    if isinstance(considered_metrics, str):
        run_history["score"] = run_history[considered_metrics]
    elif isinstance(considered_metrics, dict):
        run_history["score"] = sum(
            run_history[metric] * weight for metric, weight in considered_metrics.items()
        )
    else:
        raise ValueError("considered_metrics must be either a string or a dictionary")
    
    best_summary = run_history.iloc[run_history["score"].argmax()]
    best_summary = {f"best:{key}": val for key, val in best_summary.items()}
    
    return {
        "run_id": run.id,
        "run_name": run.name,
        "sweep_id": run.sweep.id if run.sweep else None,
        "model": run.config.get("model"),
        **run_config,
        **{metric: run_history[metric].to_list() for metric in run_history},
        **best_summary,
        "gpu_type": run.metadata.get("gpu"),
        "cpu_count": run.metadata.get("cpu_count"),
    }

batch_size = 8
records = []
futures = {}
executor = concurrent.futures.ThreadPoolExecutor(max_workers=batch_size)
runs:List[Run] = api.runs("feedr/peppermint-matrix", per_page=16, filters={"config.model": config["model"]})
run_iterator = iter(runs)
with tqdm.tqdm(total=len(runs), ncols=128) as pbar:
    while len(records) < len(runs):
        # submit new tasks if we empty slots in the batch
        while len(futures) < batch_size and len(records) + len(futures) < len(runs):
            current_runs = next(run_iterator)
            current_future = executor.submit(fetch_run_metadata, current_runs, sorting_criterion)
            futures[current_future] = current_runs

        # check for completed tasks
        finished_futures, _ = concurrent.futures.wait(futures.keys(), return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.1)
        for finished_future in finished_futures:
            finished_run = futures.pop(finished_future)
            records.append(finished_future.result())
            pbar.update(1)

# Create a Polars DataFrame from the records
experiment_runs = pl.DataFrame(records, infer_schema_length=None)
    
# # Tag run as available locally if the model files exist
local_run_ids = []
local_sweep_ids = os.listdir(f"./models/{config['model']}/")
for sweep_id in local_sweep_ids:
    local_run_ids.extend([run_id for run_id in os.listdir(f"./models/{config['model']}/{sweep_id}/")])
    
experiment_runs = experiment_runs.with_columns(
    available_locally=pl.col("run_id").is_in(local_run_ids)
)

if config["ensure_available_locally"]:
    experiment_runs = experiment_runs.filter(pl.col("available_locally") == True)

experiment_runs = experiment_runs.sort("_timestamp", descending=False)
experiment_runs = experiment_runs.with_columns(
    run_duration_second=pl.col("_runtime").list.max(),
    run_duration_minute=(pl.col("_runtime").list.max() / 60)
)
experiment_runs.select(
    pl.col("run_id"),
    pl.col("run_name"),
    pl.col("sweep_id"),
    pl.col("model"),
    pl.col("embedding_dimension"),
    pl.col("shuffle"),
    pl.col("best:epoch/epoch"),
    pl.col("best:epoch/train_loss"),
    pl.col("best:epoch/test_loss"),
    pl.col("best:epoch/test_recall@10"),
    pl.col("best:epoch/test_ndcg@10"),
)

failed to send, dropping 5 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 4 additional messages skipped
100%|█████████████████████████████████████████████████████████████████████████████████████████| 192/192 [01:29<00:00,  2.14it/s]


run_id,run_name,sweep_id,model,embedding_dimension,shuffle,best:epoch/epoch,best:epoch/train_loss,best:epoch/test_loss,best:epoch/test_recall@10,best:epoch/test_ndcg@10
str,str,str,str,i64,bool,f64,f64,f64,f64,f64
"""sm4faeql""","""electric-sweep-1""","""pjxpg4bb""","""matrix_factorization""",16,false,51.0,0.044853,0.223118,0.022528,0.087484
"""mhh2vww3""","""solar-sweep-3""","""pjxpg4bb""","""matrix_factorization""",4,true,46.0,0.092834,0.20148,0.019505,0.076231
"""im0wjtep""","""lively-sweep-2""","""pjxpg4bb""","""matrix_factorization""",32,true,26.0,0.039394,0.196034,0.022696,0.088301
"""yszchdsw""","""laced-sweep-4""","""pjxpg4bb""","""matrix_factorization""",32,true,33.0,0.035967,0.219474,0.022535,0.086142
"""gkt242cc""","""clean-sweep-5""","""pjxpg4bb""","""matrix_factorization""",256,true,11.0,0.027117,0.208644,0.020824,0.082289
"""yvji2dcx""","""rose-sweep-6""","""pjxpg4bb""","""matrix_factorization""",1024,false,1.0,0.2362,0.242782,0.023828,0.091777
"""n9r3cmud""","""warm-sweep-7""","""pjxpg4bb""","""matrix_factorization""",128,false,7.0,0.040676,0.183099,0.022164,0.085856
"""eg5qej8g""","""twilight-sweep-8""","""pjxpg4bb""","""matrix_factorization""",256,true,14.0,0.022493,0.22761,0.02086,0.082151
"""tw2l0zlb""","""fallen-sweep-9""","""pjxpg4bb""","""matrix_factorization""",1024,true,0.0,0.54582,0.310808,0.021982,0.084718


In [ ]:
experiment_runs.columns

['run_id',
 'run_name',
 'sweep_id',
 'model',
 'shuffle',
 'log_freq',
 'max_epoch',
 'batch_size',
 'random_seed',
 'store_model',
 'learning_rate',
 'early_stopping',
 'l2_regularization',
 'evaluation_cutoffs',
 'early_stopping_mode',
 'embedding_dimension',
 'early_stopping_monitor',
 'early_stopping_patience',
 'epoch/learning_rate',
 'epoch/train_map@10',
 'epoch/test_recall@2',
 'epoch/test_hitrate@2',
 'epoch/test_recall@10',
 'epoch/train_ndcg@10',
 'epoch/train_hitrate@2',
 'epoch/train_precision@10',
 'epoch/train_ndcg@2',
 'epoch/train_recall@10',
 'epoch/test_ndcg@2',
 'epoch/test_map@50',
 'epoch/test_mrr@10',
 'epoch/train_precision@2',
 'epoch/step_delta',
 'epoch/test_mrr@2',
 'epoch/test_hitrate@10',
 'epoch/test_ndcg@10',
 'epoch/test_ndcg@50',
 'epoch/train_mrr@2',
 'epoch/test_loss',
 'epoch/test_map@10',
 'epoch/train_recall@2',
 'epoch/train_mrr@10',
 'epoch/test_recall@50',
 'epoch/test_map@2',
 'epoch/epoch',
 'epoch/test_mrr@50',
 'epoch/train_mrr@50',
 'epoc

failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 3 additional messages skipped


# Parameter Comparison

## Embedding Dimension vs Regularization

In [ ]:
experiment_summary = experiment_runs.group_by("embedding_dimension", "l2_regularization").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@50").mean(),
    pl.col("best:epoch/test_ndcg@50").mean(),
).sort("embedding_dimension", "l2_regularization")
experiment_summary

embedding_dimension,l2_regularization,num_runs,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10,best:epoch/test_recall@50,best:epoch/test_ndcg@50
i64,f64,u32,f64,f64,f64,f64,f64
2,0.0,12,56.666667,0.014976,0.058856,0.054857,0.106332
2,0.00001,1,54.0,0.005822,0.024608,0.020867,0.042713
2,0.0001,1,9.0,0.002774,0.012851,0.008106,0.022531
2,0.001,2,61.0,0.000749,0.003669,0.003394,0.00848
4,0.0,9,53.555556,0.019546,0.075966,0.070581,0.131353
4,0.001,1,63.0,0.00067,0.003658,0.002944,0.008067
4,0.01,1,10.0,0.000392,0.001486,0.002421,0.005739
4,0.1,1,10.0,0.001279,0.00461,0.004109,0.009946
4,1.0,1,13.0,0.000839,0.003994,0.003417,0.009589


failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 3 additional messages skipped


In [67]:
experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_795179/2842099446.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(


embedding_dimension,0.0,0.00001,0.0001,0.001,0.01,0.1,1.0
i64,u32,u32,u32,u32,u32,u32,u32
2,12,1,1,2,null,null,null
4,9,null,null,1,1,1,1
8,15,null,1,2,1,null,1
16,18,1,2,1,null,2,null
32,20,3,1,1,null,null,3
64,11,null,null,1,1,1,1
128,11,3,3,null,1,null,null
256,12,null,null,1,1,null,4
512,15,null,2,null,1,1,2


In [ ]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(
    values=["best:epoch/epoch"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_795179/3529959983.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(


embedding_dimension,0.0,0.00001,0.0001,0.001,0.01,0.1,1.0
i64,f64,f64,f64,f64,f64,f64,f64
2,56.666667,54.0,9.0,61.0,null,null,null
4,53.555556,null,null,63.0,10.0,10.0,13.0
8,53.933333,null,7.0,63.0,8.0,null,10.0
16,44.444444,61.0,6.5,63.0,null,10.0,null
32,20.55,45.0,9.0,61.0,null,null,13.0
64,15.454545,null,null,62.0,10.0,10.0,11.0
128,12.363636,61.0,7.0,null,9.0,null,null
256,10.333333,null,null,62.0,9.0,null,12.5
512,6.466667,null,8.5,null,8.0,10.0,11.5


failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 1 additional messages skipped


In [64]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@50"]].pivot(
    values=["best:epoch/test_recall@50"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_795179/3602690543.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@50"]].pivot(


embedding_dimension,0.0,0.00001,0.0001,0.001,0.01,0.1,1.0
i64,f64,f64,f64,f64,f64,f64,f64
2,0.054857,0.020867,0.008106,0.003394,null,null,null
4,0.070581,null,null,0.002944,0.002421,0.004109,0.003417
8,0.079168,null,0.008107,0.002948,0.003977,null,0.004109
16,0.08097,0.025421,0.00828,0.003039,null,0.004109,null
32,0.080756,0.039426,0.009112,0.002863,null,null,0.004592
64,0.07939,null,null,0.003375,0.003958,0.003214,0.004336
128,0.077706,0.026922,0.008847,null,0.004234,null,null
256,0.075131,null,null,0.00161,0.003626,null,0.004465
512,0.075003,null,0.009376,null,0.003537,0.004574,0.00388


In [65]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@50"]].pivot(
    values=["best:epoch/test_ndcg@50"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_795179/4094135023.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@50"]].pivot(


embedding_dimension,0.0,0.00001,0.0001,0.001,0.01,0.1,1.0
i64,f64,f64,f64,f64,f64,f64,f64
2,0.106332,0.042713,0.022531,0.00848,null,null,null
4,0.131353,null,null,0.008067,0.005739,0.009946,0.009589
8,0.14511,null,0.020065,0.007785,0.008969,null,0.009946
16,0.147727,0.053401,0.021524,0.008105,null,0.009946,null
32,0.147998,0.079614,0.023897,0.007688,null,null,0.010116
64,0.145789,null,null,0.009162,0.007669,0.007826,0.009003
128,0.144006,0.056749,0.023081,null,0.009318,null,null
256,0.140805,null,null,0.004589,0.008446,null,0.009814
512,0.139508,null,0.023795,null,0.008749,0.009475,0.008763


# Run Duration

In [51]:
experiment_runs.group_by("embedding_dimension").agg(
    pl.col("run_duration_minute").mean()
).sort("embedding_dimension")

embedding_dimension,run_duration_minute
i64,f64
2,38.401696
4,38.107222
8,38.081819
16,37.411538
32,37.601479
64,37.70825
128,37.668705
256,37.842279
512,37.914599


In [53]:
experiment_runs.group_by("gpu_type").agg(
    pl.col("run_duration_minute").mean()
).sort("gpu_type")

gpu_type,run_duration_minute
str,f64
"""NVIDIA A10G""",37.941664


In [52]:
experiment_runs.columns

['run_id',
 'run_name',
 'sweep_id',
 'model',
 'shuffle',
 'log_freq',
 'max_epoch',
 'batch_size',
 'random_seed',
 'store_model',
 'learning_rate',
 'early_stopping',
 'l2_regularization',
 'evaluation_cutoffs',
 'early_stopping_mode',
 'embedding_dimension',
 'early_stopping_monitor',
 'early_stopping_patience',
 '_timestamp',
 'epoch/test_ndcg@10',
 'epoch/test_recall@50',
 'epoch/train_hitrate@2',
 'epoch/train_map@10',
 'epoch/train_precision@50',
 'epoch/train_mrr@10',
 'epoch/train_recall@2',
 'epoch/test_mrr@2',
 'epoch/test_map@50',
 'epoch/test_hitrate@2',
 'epoch/train_ndcg@10',
 'epoch/train_mrr@2',
 'epoch/train_hitrate@50',
 '_runtime',
 'epoch/epoch',
 'epoch/test_ndcg@2',
 'epoch/test_recall@10',
 'epoch/train_hitrate@10',
 'epoch/train_ndcg@50',
 'epoch/test_precision@10',
 'epoch/train_precision@10',
 'epoch/test_mrr@10',
 'epoch/train_recall@10',
 'epoch/train_mrr@50',
 'epoch/train_ndcg@2',
 '_step',
 'epoch/test_map@10',
 'epoch/test_hitrate@50',
 'epoch/step_del